# Machine Learning

In this file, instructions how to approach the challenge can be found.

We are going to work on different types of Machine Learning problems:

- **Regression Problem**: The goal is to predict delay of flights.
- **(Stretch) Multiclass Classification**: If the plane was delayed, we will predict what type of delay it is (will be).
- **(Stretch) Binary Classification**: The goal is to predict if the flight will be cancelled.

In [49]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime

In [2]:
#flights table
data = pd.read_csv('/Users/Prashanta/Desktop/airplane_data/flights_table(50,000_points).csv',delimiter=';')
data = data.iloc[: , 1:]

#create two samples of 10,000 points each
sample1 = data.sample(n=10000, random_state=0)
sample2 = data.sample(n=10000, random_state=1)

#Adding date time features and state
sample1['month'] = pd.DatetimeIndex(sample1['fl_date']).month
sample2['month'] = pd.DatetimeIndex(sample2['fl_date']).month
sample1['year'] = pd.DatetimeIndex(sample1['fl_date']).year
sample2['year'] = pd.DatetimeIndex(sample2['fl_date']).year
sample1['origin_city_state'] = sample1['origin_city_name'].transform(lambda x: str(x).split(',')[1])
sample2['origin_city_state'] = sample2['origin_city_name'].transform(lambda x: str(x).split(',')[1])
sample1['depart_hour'] = sample1['dep_time'].round(-2)
sample2['depart_hour'] = sample2['dep_time'].round(-2)

In [3]:
display(data.head())
display(data.shape)

,fl_date,mkt_unique_carrier,branded_code_share,mkt_carrier,mkt_carrier_fl_num,op_unique_carrier,tail_num,op_carrier_fl_num,origin_airport_id,origin,...,distance,carrier_delay,weather_delay,nas_delay,security_delay,late_aircraft_delay,first_dep_time,total_add_gtime,longest_add_gtime,no_name
0,2019-12-30,WN,WN,WN,775,WN,N255WN,775,13495,MSY,...,674.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2018-04-08,AS,AS,AS,352,AS,N464AS,352,14747,SEA,...,672.0,0.0,0.0,25.0,0.0,0.0,NaN,NaN,NaN,NaN
2,2018-11-21,DL,DL,DL,1312,DL,N912DL,1312,10994,CHS,...,259.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2019-12-19,WN,WN,WN,1967,WN,N7820L,1967,11697,FLL,...,994.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2019-09-21,UA,UA_CODESHARE,UA,3908,ZW,N420AW,3908,14574,ROA,...,530.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


(50000, 42)

## Main Task: Regression Problem

The target variable is **ARR_DELAY**. We need to be careful which columns to use and which don't. For example, DEP_DELAY is going to be the perfect predictor, but we can't use it because in real-life scenario, we want to predict the delay before the flight takes of --> We can use average delay from earlier days but not the one from the actual flight we predict.  

For example, variables **CARRIER_DELAY, WEATHER_DELAY, NAS_DELAY, SECURITY_DELAY, LATE_AIRCRAFT_DELAY** shouldn't be used directly as predictors as well. However, we can create various transformations from earlier values.

We will be evaluating your models by predicting the ARR_DELAY for all flights **1 week in advance**.

In [45]:
test_model_1 = sample1[['month','origin_city_state','depart_hour','arr_delay']]

# Droping nan value rows
test_model_1.dropna(inplace=True)

# Encode Cateogircal Values 
# First change dtype of variable to category then encode using cat.codes
test_model_1['origin_city_state'] = test_model_1['origin_city_state'].astype('category')
test_model_1['origin_city_state'] = test_model_1['origin_city_state'].cat.codes

<ipython-input-45-587ea8f238e1>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_model_1.dropna(inplace=True)
<ipython-input-45-587ea8f238e1>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_model_1['origin_city_state'] = test_model_1['origin_city_state'].astype('category')
<ipython-input-45-587ea8f238e1>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexi

In [41]:
test_model_1.head()

,month,origin_city_state,depart_hour,dep_delay,arr_delay
11841,10,37,1800.0,-7.0,-12.0
19602,7,8,1400.0,-3.0,-23.0
45519,1,6,1300.0,431.0,432.0
25747,4,49,1000.0,-4.0,10.0
42642,3,8,2000.0,140.0,135.0


In [46]:
X = test_model_1[['month','origin_city_state','depart_hour']]
y = test_model_1['arr_delay']

In [47]:
# split X and y into training and testing sets 
import sklearn.model_selection as model_selection

X_train,X_test,y_train,y_test = model_selection.train_test_split(X,y,train_size=0.75,test_size=0.25,random_state=123)

In [48]:
# import libraries
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score

#initialize and fit training dataset
Linear_Regressor = LinearRegression()
Linear_Regressor.fit(X_train, y_train)

#make prediction 
y_pred = Linear_Regressor.predict(X_test)

#prints the coefficients used for each attribute
print('coefficients:', Linear_Regressor.coef_)

#prints the R2 score for your test dataset
print('R2 Score for train data:', round(Linear_Regressor.score(X_train,y_train),3))
print('MAE:', round(mean_absolute_error(y_test,y_pred),2))
print('R2 Score2 for test data:', round(r2_score(y_test,y_pred),3))

coefficients: [-0.07959701  0.04777177  0.01045926]
R2 Score for train data: 0.012
MAE: 23.62
R2 Score2 for test data: 0.015


### Feature Engineering

Feature engineering will play a crucial role in this problems. We have only very little attributes so we need to create some features that will have some predictive power.

- weather: we can use some weather API to look for the weather in time of the scheduled departure and scheduled arrival.
- statistics (avg, mean, median, std, min, max...): we can take a look at previous delays and compute descriptive statistics
- airports encoding: we need to think about what to do with the airports and other categorical variables
- time of the day: the delay probably depends on the airport traffic which varies during the day.
- airport traffic
- unsupervised learning as feature engineering?
- **what are the additional options?**: Think about what we could do more to improve the model.

In [82]:
#Feature1
#average_dep_delay per airport
#create a dummy column and then replace values with average dep_delay per state
sample1['dep_delay_state'] = sample1['dep_delay']
sample1['dep_delay_state'] = sample1[['dep_delay_state','origin_city_state']].groupby(by='origin_city_state').transform(lambda x: x.mean())

#Feature2
#average_arr_delay per airport
#create a dummy column and then replace values with average arr_delay per state
sample1['arr_delay_state'] = sample1['arr_delay']
sample1['arr_delay_state'] = sample1[['arr_delay_state','origin_city_state']].groupby(by='origin_city_state').transform(lambda x: x.mean())

In [83]:
sample1[['month','origin_city_state','depart_hour','dep_delay','arr_delay','dep_delay_state','arr_delay_state']].head()

,month,origin_city_state,depart_hour,dep_delay,arr_delay,dep_delay_state,arr_delay_state
11841,10,PA,1800.0,-7.0,-12.0,6.660256,0.122581
19602,7,FL,1400.0,-3.0,-23.0,11.726753,5.856528
45519,1,CT,1300.0,431.0,432.0,23.268293,18.317073
25747,4,WA,1000.0,-4.0,10.0,8.521739,4.726277
42642,3,FL,2000.0,140.0,135.0,11.726753,5.856528


In [103]:
test_model_2 = sample1[['month','origin_city_state','depart_hour','dep_delay','arr_delay','dep_delay_state','arr_delay_state']]
test_model_2

,month,origin_city_state,depart_hour,dep_delay,arr_delay,dep_delay_state,arr_delay_state
11841,10,PA,1800.0,-7.0,-12.0,6.660256,0.122581
19602,7,FL,1400.0,-3.0,-23.0,11.726753,5.856528
45519,1,CT,1300.0,431.0,432.0,23.268293,18.317073
25747,4,WA,1000.0,-4.0,10.0,8.521739,4.726277
42642,3,FL,2000.0,140.0,135.0,11.726753,5.856528
...,...,...,...,...,...,...,...
25091,11,NC,1300.0,3.0,7.0,12.969838,9.566434
27853,7,MO,1500.0,15.0,21.0,8.264368,2.201149
47278,11,CO,2200.0,-4.0,-19.0,13.054264,9.497396
37020,1,SC,600.0,-10.0,-3.0,9.511111,4.955556


### Feature Selection / Dimensionality Reduction

We need to apply different selection techniques to find out which one will be the best for our problems.

- Original Features vs. PCA conponents?

### Modeling

Use different ML techniques to predict each problem.

- linear / logistic / multinomial logistic regression
- Naive Bayes
- Random Forest
- SVM
- XGBoost
- The ensemble of your own choice

### Evaluation

You have data from 2018 and 2019 to develop models. Use different evaluation metrics for each problem and compare the performance of different models.

You are required to predict delays on **out of sample** data from **first 7 days (1st-7th) of January 2020** and to share the file with LighthouseLabs. Sample submission can be found in the file **_sample_submission.csv_**

======================================================================
## Stretch Tasks

### Multiclass Classification

The target variables are **CARRIER_DELAY, WEATHER_DELAY, NAS_DELAY, SECURITY_DELAY, LATE_AIRCRAFT_DELAY**. We need to do additional transformations because these variables are not binary but continuos. For each flight that was delayed, we need to have one of these variables as 1 and others 0.

It can happen that we have two types of delays with more than 0 minutes. In this case, take the bigger one as 1 and others as 0.

### Binary Classification

The target variable is **CANCELLED**. The main problem here is going to be huge class imbalance. We have only very little cancelled flights with comparison to all flights. It is important to do the right sampling before training and to choose correct evaluation metrics.